In [1]:
import os
from tqdm import tqdm
import base64


In [2]:
images_folder = 'images'
image_files = sorted([f for f in os.listdir(images_folder) if f.endswith('.jpg') or f.endswith('.png')], key=lambda x: int(os.path.splitext(x)[0]))

In [3]:

# Função para codificar a imagem em base64
def encode_image_to_base64(image_path):
    with open(image_path, 'rb') as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Gemini

In [4]:
import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  return file

/home/rodeys/.cache/pypoetry/virtualenvs/pandd-imagens-A4lvQ6DR-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
files = [
  upload_to_gemini(f"{images_folder}/{image}", mime_type="image/jpeg") for image in tqdm(image_files)
]

100%|██████████| 2/2 [00:18<00:00,  9.48s/it]


In [25]:
model_name = "gemini-2.0-flash-thinking-exp-1219"
logs_folder = "logs/gemini"
prompt = """Extraia as informações dessa imagem, 
            Responda apenas com um json, com os campos:
            is_doc, que é um booleano que indica se o foco da imagem é um documento,
            tipo_imagem, que é uma string que indica o tipo da imagem (ex: selfie, documento, comprovante),
            info, que é um dicionário com as informações estruturadas extraídas da imagem.
            """

# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 1,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name=model_name,
  generation_config=generation_config,
)

for image in tqdm(image_files):
    log_file = os.path.join(logs_folder, os.path.splitext(image)[0] + '.log')

    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            if f.read().strip().endswith('EOF'):
                continue

    file_index = int(os.path.splitext(image)[0])
    chat_session = model.start_chat(
    history=[
        {
        "role": "user",
        "parts": [
            files[file_index],
        ],
        },
    ]
    )

    response = chat_session.send_message(prompt)
    resultado = response.text

    with open(log_file, 'a') as f:
        if os.path.getsize(log_file) > 0:   f.write('\n' * 5)
        f.write(f'model_name\n{model_name}\n\nprompt\n{prompt}\n\nresultado\n{resultado}\n\nEOF')


100%|██████████| 2/2 [00:10<00:00,  5.13s/it]


# GPT

In [4]:
from openai import OpenAI

client = OpenAI()

## GPT 4 Mini

In [32]:
model_name = "gpt-4o-mini"
logs_folder = "logs/gpt4_mini"
prompt = """Extraia as informações dessa imagem, 
            Responda apenas com um json, com os campos:
            is_doc, que é um booleano que indica se o foco da imagem é um documento,
            tipo_imagem, que é uma string que indica o tipo da imagem (ex: selfie, documento, comprovante),
            info, que é um dicionário com as informações estruturadas extraídas da imagem.
            """


for image in tqdm(image_files):
    log_file = os.path.join(logs_folder, os.path.splitext(image)[0] + '.log')

    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            if f.read().strip().endswith('EOF'):
                continue

    # Path to your image
    image_path = f"{images_folder}/{image}"

    # Getting the base64 string
    base64_image = encode_image_to_base64(image_path)

    response = client.chat.completions.create(
        model=model_name,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
    )

    
    resultado = response.choices[0].message.content

    with open(log_file, 'a') as f:
        if os.path.getsize(log_file) > 0:   f.write('\n' * 5)
        f.write(f'model_name\n{model_name}\n\nprompt\n{prompt}\n\nresultado\n{resultado}\n\nEOF')


100%|██████████| 2/2 [00:09<00:00,  4.99s/it]


## GPT4

In [33]:
model_name = "gpt-4o"
logs_folder = "logs/gpt4"
prompt = """Extraia as informações dessa imagem, 
            Responda apenas com um json, com os campos:
            is_doc, que é um booleano que indica se o foco da imagem é um documento,
            tipo_imagem, que é uma string que indica o tipo da imagem (ex: selfie, documento, comprovante),
            info, que é um dicionário com as informações estruturadas extraídas da imagem.
            """


for image in tqdm(image_files):
    log_file = os.path.join(logs_folder, os.path.splitext(image)[0] + '.log')

    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            if f.read().strip().endswith('EOF'):
                continue

    # Path to your image
    image_path = f"{images_folder}/{image}"

    # Getting the base64 string
    base64_image = encode_image_to_base64(image_path)

    response = client.chat.completions.create(
        model=model_name,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
    )

    
    resultado = response.choices[0].message.content

    with open(log_file, 'a') as f:
        if os.path.getsize(log_file) > 0:   f.write('\n' * 5)
        f.write(f'model_name\n{model_name}\n\nprompt\n{prompt}\n\nresultado\n{resultado}\n\nEOF')


100%|██████████| 2/2 [00:10<00:00,  5.11s/it]


# Groq


In [43]:
from groq import Groq

# Obter a API key do ambiente


# Inicializar o cliente Groq com a API key
client = Groq()

## groq Mini

## groq Full

In [44]:
model_name = "llama-3.2-90b-vision-preview"
logs_folder = "logs/groq"
prompt = """Extraia as informações dessa imagem, 
            Responda apenas com um json, com os campos:
            is_doc, que é um booleano que indica se o foco da imagem é um documento,
            image_type, que é uma string que indica o tipo da imagem (ex: selfie, documento, comprovante),
            info, que é um dicionário com as informações estruturadas extraídas da imagem.
            """



for image in tqdm(image_files):
    log_file = os.path.join(logs_folder, os.path.splitext(image)[0] + '.log')

    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            if f.read().strip().endswith('EOF'):
                continue

    
    image_path = f"{images_folder}/{image}"

    # Codificar a imagem em base64
    image_base64 = encode_image_to_base64(image_path)

    # Criar a solicitação de completions
    completion = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "user", "content": prompt},
                {"role": "user", "content": {"type": "image_base64", "base64": image_base64}},
            ],
            temperature=0,
            max_tokens=8192,
            top_p=1,
            stream=False,
            stop=None,
        )


    
    resultado = completion.choices[0].message

    with open(log_file, 'a') as f:
        if os.path.getsize(log_file) > 0:   f.write('\n' * 5)
        f.write(f'model_name\n{model_name}\n\nprompt\n{prompt}\n\nresultado\n{resultado}\n\nEOF')



  0%|          | 0/2 [00:00<?, ?it/s]


BadRequestError: Error code: 400 - {'error': {'message': "'messages.0' : for 'role:user' the following must be satisfied[('messages.0.content' : one of the following must be satisfied[('messages.0.content' : value must be a string) OR ('messages.0.content.1' : one of the following must be satisfied[('messages.0.content.1.type' : value is not one of the allowed values ['text']) OR ('messages.0.content.1.type' : value is not one of the allowed values ['image_url'])])])]", 'type': 'invalid_request_error'}}

# Claude

In [35]:
import anthropic

client = anthropic.Anthropic()

In [41]:
model_name = "claude-3-5-sonnet-20241022"
logs_folder = "logs/claude"
prompt = """Extraia as informações dessa imagem, 
            Responda apenas com um json, com os campos:
            is_doc, que é um booleano que indica se o foco da imagem é um documento,
            tipo_imagem, que é uma string que indica o tipo da imagem (ex: selfie, documento, comprovante),
            info, que é um dicionário com as informações estruturadas extraídas da imagem.
            """


for image in tqdm(image_files):
    log_file = os.path.join(logs_folder, os.path.splitext(image)[0] + '.log')

    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            if f.read().strip().endswith('EOF'):
                continue

    # Path to your image
    image_path = f"{images_folder}/{image}"

    image_data = encode_image_to_base64(image_path)
    media_type = "image/jpeg" 


    
    message = client.messages.create(
        model=model_name,
        max_tokens=1024*8,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": media_type,
                            "data": image_data,
                        },
                    },
                    {
                        "type": "text",
                        "text": prompt,
                    }
                ],
            }
        ],
    )
    
    resultado = message.content[0].text

    with open(log_file, 'a') as f:
        if os.path.getsize(log_file) > 0:   f.write('\n' * 5)
        f.write(f'model_name\n{model_name}\n\nprompt\n{prompt}\n\nresultado\n{resultado}\n\nEOF')


100%|██████████| 2/2 [00:12<00:00,  6.45s/it]


'{\n    "is_doc": true,\n    "tipo_imagem": "comprovante",\n    "info": {\n        "data_pagamento": "18 NOV 2024 - 04:13:32",\n        "valor": "R$ 1.544,43",\n        "data_agendada": "18 NOV 2024",\n        "favorecido": "FIDC VERDECARD",\n        "banco_destino": "BCO BRADESCO S.A.",\n        "vencimento": "20 NOV 2024",\n        "codigo_barras": "23792373044002482542012000816103399060000000000",\n        "pagador": "Adailson Carlos Colaço",\n        "banco_origem": "Nu Pagamentos S.A",\n        "agencia": "0001",\n        "conta": "43644003-3",\n        "cnpj_instituicao": "18.236.120/0001-58",\n        "id_transacao": "6738a16b-5fed-4f12-bcae-67764e78ba6f"\n    }\n}'